In [1]:
import polars as pl
import polars.selectors as cs
from glob import glob
from toolz import pipe
from functools import reduce
from operator import add, mul

In [2]:
CarData = pl.read_csv('./data/CarData_OneMonth.csv')
CarData.head()

Location,TicketID,Ticket,Status,Damage,Date / Time In,Checked-in by,Spot,Requested on,Check-Out Time,Pulled by,Delivery Time at Check-Out (d:h:m),Total Session Time (d:h:m),Pricing List Name,Amount,Tip,Collected
str,str,i64,str,i64,str,str,str,str,str,str,str,str,str,i64,i64,i64
"""Radisson Blu Mall of America""","""RbM021295""",4346,"""Returned""",0,"""9/30/2025 21:31""",null,null,"""10/2/2025 12:23""","""10/2/2025 13:04""","""Thomas Schneider""","""0h 41m""","""1d 15h 33m""","""OVERNIGHT """,68,0,68
"""Radisson Blu Mall of America""","""RbM021294""",4888,"""Returned""",0,"""9/30/2025 19:26""","""Thomas Schneider""",null,"""10/2/2025 7:16""","""10/3/2025 7:46""","""Thomas Schneider""","""1d 0h 30m""","""2d 12h 20m""","""OVERNIGHT """,102,0,102
"""Radisson Blu Mall of America""","""RbM021293""",4361,"""Returned""",0,"""9/30/2025 15:06""","""Thomas Schneider""",null,null,"""10/2/2025 9:45""",null,null,"""1d 18h 39m""","""OVERNIGHT """,68,0,68
"""Radisson Blu Mall of America""","""RbM021292""",4362,"""Returned""",0,"""9/30/2025 7:56""","""Thomas Schneider""",null,null,"""10/11/2025 15:44""",null,null,"""11d 7h 48m""","""Park-N-Stay""",null,0,0
"""Radisson Blu Mall of America""","""RbM021291""",5100,"""Returned""",0,"""9/29/2025 21:35""",null,null,"""9/30/2025 9:23""","""9/30/2025 9:27""","""Thomas Schneider""","""0h 4m""","""11h 52m""","""OVERNIGHT """,34,0,34


In [3]:
CarData_Filtered = (
    CarData.drop_nulls(pl.col('Pulled by'))
           .filter(pl.col('Pricing List Name') == 'OVERNIGHT ')
           .select(pl.col(['Location','TicketID','Status','Requested on','Pulled by',
                           'Delivery Time at Check-Out (d:h:m)', 'Total Session Time (d:h:m)','Collected']))
           .rename({'Pulled by':'Pulled_by','Requested on':'Requested_on',
                    'Delivery Time at Check-Out (d:h:m)':'Delivery_Time_at_CheckOut',
                    'Total Session Time (d:h:m)':'Total_Time_at_Hotel'
                    })
           .with_columns([
               # Extract days, hours, minutes and convert to decimal hours
               (
                ((pl.col('Delivery_Time_at_CheckOut').str.extract(r"(\d+)d", 1).cast(pl.Float64, strict=False).fill_null(0) * 24) +
                (pl.col('Delivery_Time_at_CheckOut').str.extract(r"(\d+)h", 1).cast(pl.Float64, strict=False).fill_null(0)) +
                (pl.col('Delivery_Time_at_CheckOut').str.extract(r"(\d+)m", 1).cast(pl.Float64, strict=False).fill_null(0) / 60))*60
               ).alias('DeliveryTime_Mins'),
               (
                   (pl.col('Total_Time_at_Hotel').str.extract(r"(\d+)d", 1).cast(pl.Float64, strict=False).fill_null(0) * 24 +
                   pl.col('Total_Time_at_Hotel').str.extract(r"(\d+)h", 1).cast(pl.Float64, strict=False).fill_null(0) +
                   pl.col('Total_Time_at_Hotel').str.extract(r"(\d+)m", 1).cast(pl.Float64, strict=False).fill_null(0) / 60)*60
               ).alias('Total_Time_at_Hotel_Mins'),
               (
                   pl.col('Requested_on').str.strptime(pl.Date, format="%m/%d/%Y %H:%M").dt.strftime("%Y-%m-%d")
               ).alias('Requested_on')
           ])
           .filter(pl.col('DeliveryTime_Mins') <= 60)
           .drop(['Delivery_Time_at_CheckOut','Total_Time_at_Hotel'])
)
CarData_Filtered.head()

Location,TicketID,Status,Requested_on,Pulled_by,Collected,DeliveryTime_Mins,Total_Time_at_Hotel_Mins
str,str,str,str,str,i64,f64,f64
"""Radisson Blu Mall of America""","""RbM021295""","""Returned""","""2025-10-02""","""Thomas Schneider""",68,41.0,2373.0
"""Radisson Blu Mall of America""","""RbM021291""","""Returned""","""2025-09-30""","""Thomas Schneider""",34,4.0,712.0
"""Radisson Blu Mall of America""","""RbM021290""","""Returned""","""2025-09-30""","""Thomas Schneider""",34,1.0,528.0
"""Radisson Blu Mall of America""","""RbM021289""","""Returned""","""2025-09-30""","""Thomas Schneider""",34,7.0,805.0
"""Radisson Blu Mall of America""","""RbM021288""","""Returned""","""2025-09-30""","""Thomas Schneider""",34,5.0,729.0


In [4]:
CarData_Filtered_check = (
    CarData_Filtered
    .group_by('Pricing List Name')
)

### A few Notes:

#### Pull Times:

- These pull numbers by employee will be inaccurate, phones at the Blu are hardly ever logged in to the correct person, would be better to look at Employee clock times and look at the correlation between a certain employee and retrieval times druing their shifts. Individual results will be hard to gather unless an employee is working by themself. 
- Useful to keep an employees on counter as well.

#### How to Provide more insight to if a time is better:

- Find the amount of people who are checked in at a certain time. Could do this with some sort of counter by date/time, If someone is checked in they count towards that counter for the day, once they check out they -1 from the counter.

- If possible include events going on at the hotel, would be nice to have a dataset with WHEN the event was, how MANY people were expected, how many CARS expected.

##### Modeling how many employees are needed:

- Response Variable: employees
- Getting events data would help greatly with this as well. 
- Number of Check Ins important
- Retreival Times (However I'm doubtful of how important this will be due to issue of sitting on the drive)
- What can be considered a "normal" retrieval time? Is there data on when the Car has been designated as "ready" to the customer?
- 



In [5]:
import plotly.express as px

# Create box plot to show distributions and outliers
fig = px.box(
    CarData_Filtered,
    x='Pulled_by',
    y='DeliveryTime_Mins',
    title='Delivery Time Distribution by Valet (with Outliers)',
    labels={'Pulled_by': 'Valet', 'DeliveryTime_Mins': 'Delivery Time (Minutes)'},
    points='all',  # Show all individual points
    hover_data=['TicketID', 'Requested_on'],
    color='Pulled_by'  # Different color for each valet
)

# Remove grid lines and improve appearance
fig.update_layout(
    showlegend=False,  # Remove legend since x-axis already shows valet names
    xaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=2,
        linecolor='black'
    ),
    yaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=2,
        linecolor='black'
    ),
    plot_bgcolor='white',
    font=dict(size=12)
)

fig.show()

In [6]:
import plotly.express as px

# Aggregate by date and calculate cumulative sum
cumsum_data = (
    CarData_Filtered
    .group_by('Requested_on')
    .agg(pl.col('Collected').sum().alias('Daily_Total'))
    .sort('Requested_on')
    .with_columns(
        pl.col('Daily_Total').cum_sum().alias('Cumulative_Collected')
    )
)

# Create line plot
fig = px.line(
    cumsum_data,
    x='Requested_on',
    y='Cumulative_Collected',
    title='Cumulative Amount Collected Over Time',
    labels={'Requested_on': 'Date Requested', 'Cumulative_Collected': 'Cumulative Amount Collected ($)'},
    markers=True
)

# Improve appearance
fig.update_layout(
    xaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=2,
        linecolor='black'
    ),
    yaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=2,
        linecolor='black'
    ),
    plot_bgcolor='white',
    font=dict(size=12)
)

fig.show()